In [2]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [3]:

# Path to dataset (download FER2013 from Kaggle and extract)
train_dir = "train"
val_dir   = "test"


In [2]:
!pip install tensorflow

In [4]:

# Data augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48,48),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical"
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(48,48),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical"
)


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [5]:

# Build CNN Model
model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))  # 7 emotions

# Compile
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30
)

# Save model
model.save("emotion_detection_model.h5")
print("Model saved as emotion_detection_model.h5")


C:\Users\Admin\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Admin\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 350s 763ms/step - accuracy: 0.2097 - loss: 2.1900 - val_accuracy: 0.2432 - val_loss: 1.9701
Epoch 2/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 133s 297ms/step - accuracy: 0.2524 - loss: 1.8064 - val_accuracy: 0.2655 - val_loss: 1.7702
Epoch 3/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 132s 295ms/step - accuracy: 0.2675 - loss: 1.7766 - val_accuracy: 0.3526 - val_loss: 1.6612
Epoch 4/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 130s 288ms/step - accuracy: 0.2807 - loss: 1.7509 - val_accuracy: 0.3530 - val_loss: 1.6204
Epoch 5/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 133s 297ms/step - accuracy: 0.3014 - loss: 1.7166 - val_accuracy: 0.2921 - val_loss: 1.7222
Epoch 6/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 137s 305ms/step - accuracy: 0.3124 - loss: 1.6985 - val_accuracy: 0.4058 - val_loss: 1.5198
Epoch 7/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 141s 313ms/step - accuracy: 0.3330 - loss: 1.6650 - val_accuracy: 0.3778 - val_loss: 1.5789
Epoch 8/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 138s 307ms/step - accuracy: 0.3490 -

Model saved as emotion_detection_model.h5


In [1]:

# ==============================
# STEP 2: Real-time Detection + Music Suggestion
# ==============================

import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load trained CNN model
model = load_model("emotion_detection_model.h5")

# Emotion labels
emotion_labels = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

# Music suggestions for each emotion
emotion_songs = {
    "Angry":    ["Lose Yourself - Eminem", "Numb - Linkin Park"],
    "Disgust":  ["Boulevard of Broken Dreams - Green Day"],
    "Fear":     ["Fix You - Coldplay"],
    "Happy":    ["Happy - Pharrell Williams", "Uptown Funk - Bruno Mars"],
    "Sad":      ["Someone Like You - Adele", "Let Her Go - Passenger"],
    "Surprise": ["Viva La Vida - Coldplay"],
    "Neutral":  ["Shape of You - Ed Sheeran"]
}

# Haarcascade face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48,48))
        roi_gray = roi_gray.astype("float")/255.0
        roi_gray = np.expand_dims(roi_gray, axis=0)
        roi_gray = np.expand_dims(roi_gray, axis=-1)

        # Predict emotion
        prediction = model.predict(roi_gray, verbose=0)
        emotion_index = np.argmax(prediction)
        emotion = emotion_labels[emotion_index]
        
        # Suggest a song
        suggested_song = np.random.choice(emotion_songs[emotion])
        
        # Draw
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(frame, f"{emotion} -> {suggested_song}", 
                    (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
    
    cv2.imshow("Emotion Detection with Music Suggestion", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import matplotlib.pyplot as plt


# If you already have the history object from training, just plot:
# (Replace `history` below with your actual history variable)

# Accuracy graph
plt.figure(figsize=(8,6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()

# Loss graph
plt.figure(figsize=(8,6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'history.pkl'